## ChatGPT API with Pyhon 快速入门
1. 创建API Key
2. 基本框架 + 参数理解
3. 实现连续提问
4. System,Assistant,User 三种角色的定义和使用

In [1]:
import os
import openai
from configparser import RawConfigParser

CONF_FILE = os.path.abspath(os.path.dirname('.')) + '/conf.ini'

In [2]:
# load OpenAI API Key
config = RawConfigParser()
config.read(CONF_FILE)
openai.api_key = config['openai']['chatgpt_key']

### 极简框架
+ 粘贴复制 api key
+ 两个重要基本参数：**model** 和 **messages**，更多参数如 **max_token, temperature**, 参考 [here](https://platform.openai.com/docs/api-reference/chat/create)
+ 模型概览，参考 [here](https://platform.openai.com/docs/models/overview)
+ role有三种角色定义: system，assistant, user, 参考 [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

In [3]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", 
  messages=[
    {"role": "user", "content": "Tell me a joke about bike."}
  ]
)

print(completion.choices[0].message.content)



Why did the bicycle fall over? Because it was two-tired!


### System role的定义
+ system是更高层的角色，意思就是TA可以控制assistant的行为，比如让assistant成为一个英译中的角色

In [7]:
messages = [{"role": "system", "content" : "You are a great assistant that help translate English to Chinese."}]

In [8]:
content = input("玩转数据老六: ") #定义“你”的昵称+输入内容
messages.append({"role": "user", "content": content})

completion = openai.ChatCompletion.create(
  model = "gpt-3.5-turbo",
  messages = messages
)

ai_rsp = completion.choices[0].message.content
print(f"AI小七: {ai_rsp}")

玩转数据老六:  what is bike?


AI小七: 自行车。


#### Q：如何实现连续提问？
+ 可使用 while 语句

In [10]:
messages = [{"role": "system", "content" : "You are a great assistant."}]
while True:

    content = input("玩转数据老六: ")
    messages.append({"role": "user", "content": content})

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = messages
    )

    ai_rsp = completion.choices[0].message.content
    print(f"AI小七: {ai_rsp}")

玩转数据老六:  who is yaoming?


AI小七: Yao Ming is a retired Chinese professional basketball player who played for the Houston Rockets in the National Basketball Association (NBA) from 2002 to 2011. He is considered one of the greatest Asian basketball players of all time and was inducted into the Naismith Memorial Basketball Hall of Fame in 2016.


玩转数据老六:  how tall is he?


AI小七: Yao Ming is a retired Chinese professional basketball player who played for the Houston Rockets in the NBA. He is 7 feet 6 inches tall (2.29 meters), making him one of the tallest basketball players in history.


玩转数据老六:  where is he from?


AI小七: Yao Ming is a retired Chinese professional basketball player who played in the National Basketball Association (NBA) for the Houston Rockets from 2002 to 2011. He is 7 feet 6 inches (2.29 meters) tall. He was born in Shanghai, China, and played for the Chinese national basketball team before joining the NBA.


KeyboardInterrupt: Interrupted by user

从上面的连续提问可以看到存在一个问题：历史对话会不断保存，显示冗余，可以通过assistant role来解决这个问题

### Assistant Role
+ 帮助消除冗余显示

In [11]:
messages = [{"role": "system", "content" : "You are a great assistant."}]
while True:

    content = input("玩转数据老六: ")
    messages.append({"role": "user", "content": content})

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = messages
    )

    ai_rsp = completion.choices[0].message.content
    print(f"AI小七: {ai_rsp}")
    print("")
    messages.append({"role": "assistant", "content": ai_rsp})

玩转数据老六:  who is yaoming?


AI小七: Yao Ming is a retired Chinese professional basketball player. He played for the Houston Rockets in the NBA and was one of the most popular and successful athletes from China.



玩转数据老六:  how tall is he?


AI小七: Yao Ming is 7 feet 6 inches (2.29 meters) tall.



玩转数据老六:  where is he from?


AI小七: Yao Ming is from Shanghai, China.



KeyboardInterrupt: Interrupted by user

## 更多信息，欢迎关注
+ 小红书：[数据六总](https://www.xiaohongshu.com/user/profile/5e67aeb60000000001007756?xhsshare=CopyLink&appuid=5e67aeb60000000001007756&apptime=1678807494)
+ 知乎：[六总](https://www.zhihu.com/people/yuntongxue)